In [ ]:
from google.colab import auth, drive
auth.authenticate_user()

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import pandas as pd
import os
import random
from pathlib import Path
from pandas.core.common import SettingWithCopyWarning
import warnings
import yaml
from google.cloud import bigquery

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

mimicdir = '/content/gdrive/My Drive/hst953_hw1/mimic_data'
Path(mimicdir).mkdir(parents = True, exist_ok = True)

random.seed(42)
np.random.seed(42)

In [ ]:
project_id='CHANGE_THIS'

def run_query(query):
    return pd.io.gbq.read_gbq(query, project_id=project_id, dialect="standard")

In [ ]:
denquery = """
SELECT ie.subject_id, ie.hadm_id, ie.icustay_id
, pat.gender
, adm.admittime, adm.dischtime, adm.diagnosis
, DATETIME_DIFF(ie.intime, pat.dob, YEAR) as age
, adm.ethnicity, adm.ADMISSION_TYPE, adm.language, adm.insurance
, adm.hospital_expire_flag
, CASE when adm.deathtime between ie.intime and ie.outtime THEN 1 ELSE 0 END AS mort_icu
, DENSE_RANK() OVER (PARTITION BY adm.subject_id ORDER BY adm.admittime) AS hospstay_seq
, CASE
    WHEN DENSE_RANK() OVER (PARTITION BY adm.subject_id ORDER BY adm.admittime) = 1 THEN 1
    ELSE 0 END AS first_hosp_stay
, ie.intime, ie.outtime
, ie.FIRST_CAREUNIT as first_careunit
, DENSE_RANK() OVER (PARTITION BY ie.hadm_id ORDER BY ie.intime) AS icustay_seq

, CASE
    WHEN DENSE_RANK() OVER (PARTITION BY ie.hadm_id ORDER BY ie.intime) = 1 THEN 1
    ELSE 0 END AS first_icu_stay

FROM `physionet-data.mimiciii_clinical.icustays` ie
INNER JOIN `physionet-data.mimiciii_clinical.admissions` adm
    ON ie.hadm_id = adm.hadm_id
INNER JOIN `physionet-data.mimiciii_clinical.patients` pat
    ON ie.subject_id = pat.subject_id
WHERE adm.has_chartevents_data = 1
ORDER BY ie.subject_id, adm.admittime, ie.intime;
"""

den = run_query(denquery)

In [ ]:
den['los_icu_hr'] = (den.outtime - den.intime).astype('timedelta64[h]')
den = den[(den.los_icu_hr >= 48)]
den = den[(den.age<300)]
den.drop('los_icu_hr', 1, inplace = True)
den = den[(den.first_hosp_stay == 1) & (den.first_icu_stay == 1)]
den = den[~(den['first_careunit'].isin(['PICU', 'NICU']))]

den.ethnicity = den.ethnicity.str.lower()
den.ethnicity.loc[(den.ethnicity.str.contains('^white'))] = 'white'
den.ethnicity.loc[(den.ethnicity.str.contains('^black'))] = 'black'
den.ethnicity.loc[(den.ethnicity.str.contains('^hisp')) | (den.ethnicity.str.contains('^latin'))] = 'hispanic'
den.ethnicity.loc[(den.ethnicity.str.contains('^asia'))] = 'asian'
den.ethnicity.loc[~(den.ethnicity.str.contains('|'.join(['white', 'black', 'hispanic', 'asian'])))] = 'other'

den.drop(['diagnosis', 'hospstay_seq', 'icustay_seq', 'outtime', 'first_careunit', 'first_hosp_stay', 'first_icu_stay'], 1, inplace =True)

def map_lang(x):
    if x == 'ENGL':
        return 'English'
    if pd.isnull(x):
        return 'Missing'
    return 'Other'
den['language'] = den['language'].apply(map_lang)

In [ ]:
with open('./icd9_codes.yml', 'r') as f:
    ccs = pd.DataFrame.from_dict(yaml.load(f)).T

icd = run_query('select * from `physionet-data.mimiciii_clinical.diagnoses_icd`')
icd.columns = map(str.lower, icd.columns)
icd = (icd.groupby(['subject_id','hadm_id'], as_index = False)
       .agg({'icd9_code': list})
      .merge(den[['subject_id', 'hadm_id']], on = ['subject_id', 'hadm_id'], how = 'inner')
      .explode('icd9_code'))

icd = (pd.merge(icd, (ccs[ccs['use_in_benchmark']]
                     .reset_index()
                     .explode('codes')), left_on = 'icd9_code', right_on = 'codes', how = 'inner')
      .rename(columns = {'index': 'name'}).drop(columns = ['use_in_benchmark', 'id', 'type', 'codes']))

targets = list(ccs[ccs['use_in_benchmark']].index)
target_df = icd.pivot_table(index = ['subject_id', 'hadm_id'], columns = 'name', values = 'icd9_code', aggfunc = lambda x: x)
target_df = (target_df.where(pd.isnull(target_df), 1)
             .fillna(0)
             .reset_index())

target_df['any acute'] = target_df[ccs[(ccs['use_in_benchmark']) & (ccs['type'] == 'acute')].index].any(axis = 1).astype(int)
target_df['any chronic'] = target_df[ccs[(ccs['use_in_benchmark']) & (ccs['type'] == 'chronic')].index].any(axis = 1).astype(int)

mapping = pd.read_csv('./mapping.csv').set_index('before')['after'].to_dict()
target_df = target_df.rename(columns = mapping)
den = pd.merge(den, target_df, how = 'left', on = ['subject_id', 'hadm_id'])

for col in mapping.values():
    den[col] = den[col].fillna(0)

In [ ]:
notesquery = """
select row_id as note_id, subject_id, hadm_id, chartdate, charttime, category, text
from `physionet-data.mimiciii_notes.noteevents`
where category in ('Discharge summary', 'Nursing', 'Nursing/other')
"""

notes = run_query(notesquery)
notes = notes[(notes.hadm_id.isin(den.hadm_id))]
notes['text'] = notes['text'].str.lower().apply(str.strip).fillna('')
notes['text'] = (notes['text'].str.replace(r'(-){2,}|_{2,}|={2,}', '')
                 .str.replace(r'[0-9]+\.', '')
                 .str.replace(r'\[(.*?)\]', '')
                 .str.replace(r'dr\.', 'doctor')
                 .str.replace(r'm\.d\.', 'md')
                 .str.replace(r'admission date:', '')
                 .str.replace(r'discharge date:', '')
                 .str.replace(r'\n', ' ')
                 .str.replace(r'\r', ' ')
                )

den = den[den['subject_id'].isin(notes['subject_id'])] #drop people with no notes

msk = np.random.rand(len(den)) < 0.7
den['train'] = np.where(msk, 1, 0)
den.to_hdf(os.path.join(mimicdir, 'cohort.h5'), 'cohort', index = False)
notes.to_hdf(os.path.join(mimicdir, 'notes.h5'), 'notes', index = False)